In [9]:
import boto3
import json
import arxiv
import urllib
import pdfx

In [10]:
search = arxiv.Search(
    query = "computer science &  ai",
    # id_list=["1605.08386v1"]
    max_results = 1,
    sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
    print ("Titre :", result.title)
    print ("Auteurs :", result.authors)
    print ("Lien :", result.pdf_url)

Titre : Unsupervised Prompt Learning for Vision-Language Models
Auteurs : [arxiv.Result.Author('Tony Huang'), arxiv.Result.Author('Jack Chu'), arxiv.Result.Author('Fangyun Wei')]
Lien : http://arxiv.org/pdf/2204.03649v1


In [11]:
response = urllib.request.urlopen(result.pdf_url)    
file = open("Test" + ".pdf", 'wb')
file.write(response.read())
file.close()

filename = 'Test.pdf'

## Lecture du PDF

### Pour mettre un PDF au choix, décommenter et indiquer le path

In [12]:
# filename = '...'

In [13]:
pdf = pdfx.PDFx(filename)
metadata = pdf.get_metadata()
references_dict = pdf.get_references_as_dict()
text = pdf.get_text()

In [14]:
def after_references(mypdftext): 
    keyword1 = 'References'
    keyword2 = 'REFERENCES'
    keyword3 = 'R EFERENCES'
    keyword4 = 'Reference'
    keyword5='[1]' 

    if keyword1 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword1)
    elif keyword2 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword2)
    elif keyword3 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword3)
    elif keyword4 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword4)
    elif keyword5 in mypdftext :
            before_keyword, keyword, after_keyword = mypdftext.partition(keyword5)
    else:
        after_keyword = mypdftext[:10000]
    return after_keyword

#All references in a variable

references = after_references(text)

In [15]:
print (references[:500])



1. Automated flower classification over a large number of classes. In: 2008 Sixth Indian
Conference on Computer Vision, Graphics & Image Processing. pp. 722–729. IEEE
(2008)

2. Bossard, L., Guillaumin, M., Van Gool, L.: Food-101–mining discriminative com-
ponents with random forests. In: European conference on computer vision. pp.
446–461. Springer (2014)

3. Chen, T., Kornblith, S., Norouzi, M., Hinton, G.: A simple framework for con-
trastive learning of visual representations. In: Internat


# AWS Comprehend

## Indiquez vos identifiants d'access

In [16]:
ACCESS_ID='...'
ACCESS_KEY='...'
session_token='...'

#### Access Personnel (A laisser en commentaire)

In [17]:
# Access Personnel

# import toml

# access = toml.load('aws_access.toml')

# ACCESS_ID=access['default']['aws_access_key_id']
# ACCESS_KEY=access['default']['aws_secret_access_key']
# session_token=access['default']['aws_session_token']

In [18]:

comprehend = boto3.client(service_name='comprehend', region_name='us-east-1',aws_access_key_id=ACCESS_ID,
                                            aws_secret_access_key= ACCESS_KEY,aws_session_token=session_token
)


results = comprehend.detect_entities(Text=references[:3000], LanguageCode='en')

clean_results = []
i=0
for r in results['Entities']:
    line={}
    if r["Type"] == "PERSON" and r["Score"] > 0.95:
        line['id'] = i
        line['Person']= r['Text']
        clean_results.append(line)
        i += 1

print(json.dumps(clean_results))


[{"id": 0, "Person": "Bossard, L."}, {"id": 1, "Person": "Chen, T."}, {"id": 2, "Person": "Kornblith, S."}, {"id": 3, "Person": "Norouzi, M."}, {"id": 4, "Person": "Hinton, G."}, {"id": 5, "Person": "Chen, X."}, {"id": 6, "Person": "Kokkinos, I."}, {"id": 7, "Person": "Mohamed, S."}, {"id": 8, "Person": "Deng, J."}, {"id": 9, "Person": "Tony Huang1\u2217"}, {"id": 10, "Person": "Jack Chu1\u2217"}, {"id": 11, "Person": "Dosovitskiy, A."}, {"id": 12, "Person": "Beyer, L."}, {"id": 13, "Person": "Kolesnikov, A."}, {"id": 14, "Person": "Weissenborn, D."}, {"id": 15, "Person": "Gelly, S."}, {"id": 16, "Person": "Buchatskaya"}, {"id": 17, "Person": "Doer-\nsch"}, {"id": 18, "Person": "Avila Pires"}]
